In [1]:
#general 
import pandas as pd
import numpy as np
import os
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split

#warnings 
import warnings 
warnings.filterwarnings("ignore")

#for RNN
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model

import ast

Create Sentences for testing RNNS

In [2]:
word_df = pd.read_pickle("pos_tag_word_list")
sent_df = pd.read_csv("sample sent idea.csv")

Develop Benchmarks

In [3]:
def benchmark_string(sents):
    list_of_strings = []

    for word_list in sents:
        list_of_strings.append(' '.join(word_list))
    return list_of_strings

In [4]:
sent_df['list'] = sent_df['list'].apply(ast.literal_eval)
benchmark_sentences = sent_df['list'].tolist()
all_benchmarks = benchmark_string(benchmark_sentences)

In [5]:
loaded_model = tf.keras.models.load_model("C:/Users/danie/Desktop/RNN_2012")

In [6]:
X_b = all_benchmarks
x_arrayb = np.asarray(X_b)
y_predb = loaded_model.predict(X_b)
Yb = pd.DataFrame(y_predb)
Yb.reset_index(inplace=True, drop=True)
Yb.set_axis(['0 Class Benchamark', '1 Class Benchmark'], axis='columns', inplace=True)
sent_df = pd.concat([sent_df, Yb], axis=1)

5/5 [==============================] - 6s 50ms/step


Test RNN model

#NOUNS

In [7]:
noun_list = word_df[(word_df.pos_tag_short == "N")]["feature"].values.tolist()
verb_list = word_df[(word_df.pos_tag_short == "V")]["feature"].values.tolist()
adverb_list = word_df[(word_df.pos_tag_short == "R")]["feature"].values.tolist()
adjective_list = word_df[(word_df.pos_tag_short == "J")]["feature"].values.tolist()

noun_sent = sent_df[(sent_df['POS tag'] == "Noun")]["replacement string"].values.tolist()
verb_sent = sent_df[(sent_df['POS tag'] == "Verb")]["replacement string"].values.tolist()
adjective_sent = sent_df[(sent_df['POS tag'] == "Adjective")]["replacement string"].values.tolist()
adverb_sent = sent_df[(sent_df['POS tag'] == "Adverb")]["replacement string"].values.tolist()

adverb_len = len(adverb_sent)
verb_len = len(verb_sent)
noun_len = len(noun_sent)
verb_len = len(verb_sent)

In [8]:
def word_list(words, sents):
    new_token_sents = []
    for word in words:
        temp_sents = []
        temp_sent_2 = []
        for sent in sents:
            new_sent = ' '.join([word if w == 'UNK' else w for w in sent])
            temp_sents.append([new_sent, sent])
        new_token_sents.extend(temp_sents)
    return new_token_sents

sent_n = pd.DataFrame(word_list(noun_list, noun_sent))
sent_j = pd.DataFrame(word_list(adjective_list, adjective_sent))
sent_a = pd.DataFrame(word_list(adverb_list, adverb_sent))
sent_v = pd.DataFrame(word_list(verb_list, verb_sent))

In [9]:
x_array_n = np.asarray(sent_n[0])
x_array_j = np.asarray(sent_j[0])
x_array_v = np.asarray(sent_v[0])
x_array_a = np.asarray(sent_a[0])


y_pred_n = loaded_model.predict(x_array_n)
y_pred_j = loaded_model.predict(x_array_j)
y_pred_v = loaded_model.predict(x_array_v)
y_pred_a = loaded_model.predict(x_array_a)

1/1 [==============================] - 0s 65ms/step


In [10]:
#You need to change the num_repititons to the number of sentences in the function

def labels(word_list, word_len):
    num_repetitions = word_len
    repeated_words = []
    for word in word_list:
        for _ in range(num_repetitions):
            repeated_words.append(word)
    return repeated_words

noun_labels = pd.DataFrame(labels(noun_list, noun_len))
adjective_labels = pd.DataFrame(labels(adjective_list, noun_len))
verb_labels = pd.DataFrame(labels(verb_list, noun_len))
adverb_labels = pd.DataFrame(labels(adverb_list, noun_len))



In [11]:
Y_n = pd.DataFrame(y_pred_n)
Y_j = pd.DataFrame(y_pred_j)
Y_v = pd.DataFrame(y_pred_v)
Y_a = pd.DataFrame(y_pred_a)

result_n = pd.concat([noun_labels, Y_n,sent_n], axis=1)
result_j = pd.concat([adjective_labels, Y_j,sent_j], axis=1)
result_v = pd.concat([verb_labels, Y_v,sent_v], axis=1)
result_a = pd.concat([adverb_labels, Y_a, sent_a], axis=1)

result_n.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_j.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_v.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)
result_a.set_axis(['feature', '0 Class', '1 Class', 'RNN Sentences', 'replacement string'], axis='columns', inplace=True)

In [12]:
result = pd.concat([result_a, result_v, result_j, result_n])

In [13]:
result_with_source = pd.merge(result, word_df, on='feature', how='inner')

In [14]:
df = pd.merge(result_with_source, sent_df, on='replacement string', how='inner')

In [15]:
df['Class 0 Differential'] = df['0 Class'] - df['0 Class Benchamark']
df['Class 1 Differential'] = df['1 Class'] - df['1 Class Benchmark']


In [17]:
df

,feature,0 Class,1 Class,RNN Sentences,replacement string,source,pos_tag,pos_tag_short,snet id,list,sent,POS tag,word num,identified word,0 Class Benchamark,1 Class Benchmark,Class 0 Differential,Class 1 Differential
0,friendly,-1.310452,1.817012,"[ ' s a y ' , ' g u y ' , ' h e a r ' , ...","['say', 'guy', 'hear', 'sing', 'think', 'moyle...",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7.0,ago,-0.034972,0.058603,-1.275480,1.758409
1,friendly,-1.310452,1.817012,"[ ' s a y ' , ' g u y ' , ' h e a r ' , ...","['say', 'guy', 'hear', 'sing', 'think', 'moyle...",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7.0,ago,-0.034972,0.058603,-1.275480,1.758409
2,friendly,-1.310452,1.817012,"[ ' s a y ' , ' g u y ' , ' h e a r ' , ...","['say', 'guy', 'hear', 'sing', 'think', 'moyle...",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7.0,ago,-0.034972,0.058603,-1.275480,1.758409
3,friendly,-1.310452,1.817012,"[ ' s a y ' , ' g u y ' , ' h e a r ' , ...","['say', 'guy', 'hear', 'sing', 'think', 'moyle...",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7.0,ago,-0.034972,0.058603,-1.275480,1.758409
4,friendly,-1.310452,1.817012,"[ ' s a y ' , ' g u y ' , ' h e a r ' , ...","['say', 'guy', 'hear', 'sing', 'think', 'moyle...",communality,RB,R,7551,"[say, guy, hear, sing, think, moyles, year, ago]","He said: ""Some guys heard me singing Tom Jones...",Adverb,7.0,ago,-0.034972,0.058603,-1.275480,1.758409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55467,slender,-2.492111,3.155214,"[ ' t e r r i f y ' , ' U N K ' , ' p l a ...","['terrify', 'UNK', 'play', 'well', 'together',...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1.0,pace,2.273282,-2.246118,-4.765393,5.401331
55468,handsome,-2.492111,3.155214,"[ ' t e r r i f y ' , ' U N K ' , ' p l a ...","['terrify', 'UNK', 'play', 'well', 'together',...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1.0,pace,2.273282,-2.246118,-4.765393,5.401331
55469,fat,-2.492111,3.155214,"[ ' t e r r i f y ' , ' U N K ' , ' p l a ...","['terrify', 'UNK', 'play', 'well', 'together',...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1.0,pace,2.273282,-2.246118,-4.765393,5.401331
55470,thin,-2.492111,3.155214,"[ ' t e r r i f y ' , ' U N K ' , ' p l a ...","['terrify', 'UNK', 'play', 'well', 'together',...",Apperance,NN,N,9512,"[terrify, pace, play, well, together, also, sc...",Aaron Lennon terrified the Croats with his pac...,Noun,1.0,pace,2.273282,-2.246118,-4.765393,5.401331


In [16]:
df.groupby(by=['source']).mean()

,0 Class,1 Class,snet id,word num,0 Class Benchamark,1 Class Benchmark,Class 0 Differential,Class 1 Differential
source,,,,,,,,
Apperance,-0.263607,0.666855,9109.920000,2.054795,0.725785,-0.631067,-0.989392,1.297922
Violence,-0.262859,0.666209,9113.810778,2.066480,0.725125,-0.630511,-0.987984,1.296719
agency,-0.260481,0.664143,9126.247119,2.108044,0.725425,-0.631493,-0.985907,1.295637
communality,-0.242605,0.640061,8795.695730,2.401367,0.751181,-0.664689,-0.993787,1.304750
power,-0.263783,0.667356,9114.352916,2.067544,0.724620,-0.629988,-0.988404,1.297344
